# 💊 PharmaX Analysis - Medical Prescription & Reimbursement (France, 2023)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
import os

In [ ]:
# Load and prepare data
file_path = 'data/open_medic_2023.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1', sep=';')
df.rename(columns={
    'l_cip13': 'medication',
    'BOITES': 'boxes',
    'REM': 'reimbursement',
    'age': 'age',
    'sexe': 'gender',
    'BEN_REG': 'region'
}, inplace=True)
df = df[['medication', 'boxes', 'reimbursement', 'age', 'gender', 'region']]
df.dropna(inplace=True)

In [ ]:
# Convert reimbursement to float (handle commas)
df['reimbursement'] = df['reimbursement'].astype(str).str.replace(',', '.')
df['reimbursement'] = pd.to_numeric(df['reimbursement'], errors='coerce')
print("Sample reimbursement values:", df['reimbursement'].head())
print("Total reimbursement sum:", df['reimbursement'].sum())

In [ ]:
# Map region codes to names
region_map = {
    11: 'Île-de-France', 24: 'Centre-Val de Loire', 27: 'Bourgogne-Franche-Comté',
    28: 'Normandie', 32: 'Hauts-de-France', 44: 'Grand Est',
    52: 'Pays de la Loire', 53: 'Bretagne', 75: 'Nouvelle-Aquitaine',
    76: 'Occitanie', 84: 'Auvergne-Rhône-Alpes', 93: 'PACA', 94: 'Corse'
}
df['region_name'] = df['region'].map(region_map)

In [ ]:
# Top 10 reimbursed medications
top_reimbursed = df.groupby('medication')['reimbursement'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 5))
sns.barplot(x=top_reimbursed.values, y=top_reimbursed.index, palette='crest')
plt.title('Top 10 Reimbursed Medications (France, 2023)')
plt.xlabel('Total Reimbursement (€)')
plt.ylabel('Medication')
plt.tight_layout()
plt.savefig('assets/top_10_reimbursed_meds.png')
plt.show()

In [ ]:
# Reimbursement by region
reimbursement_summary = df.groupby('region_name')['reimbursement'].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=reimbursement_summary.values, y=reimbursement_summary.index, palette='rocket')
plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{x/1e6:.1f}M€'))
plt.title('Total Reimbursement by Region (France, 2023)')
plt.xlabel('Reimbursement Amount (in millions €)')
plt.ylabel('Region')
plt.tight_layout()
plt.savefig('assets/reimbursement_by_region.png')
plt.show()

In [ ]:
# Prescriptions by region
region_summary = df.groupby('region_name')[['boxes', 'reimbursement']].sum().sort_values(by='boxes', ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=region_summary['boxes'], y=region_summary.index, palette='mako')
plt.gca().xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{int(x/1e6)}M'))
plt.title('Total Prescriptions by Region (France, 2023)')
plt.xlabel('Number of Boxes (in millions)')
plt.ylabel('Region')
plt.tight_layout()
plt.savefig('assets/prescriptions_by_region.png')
plt.show()

In [ ]:
# Top 10 most prescribed medications
top_meds = df.groupby('medication')['boxes'].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 5))
sns.barplot(x=top_meds.values, y=top_meds.index, palette='crest')
plt.title('Top 10 Most Prescribed Medications (France, 2023)')
plt.xlabel('Number of Boxes')
plt.ylabel('Medication')
plt.tight_layout()
plt.savefig('assets/top_10_meds.png')
plt.show()